In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt

import ast

import xgboost as xgb

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

In [3]:
def xgb_LOSO_runner(data, params, train_masks, validate_masks, test_masks, test=False):
    feats = data.drop(['Subject', 'Activity', 'Label'], axis=1)
    labels = data.Label
    
    # metrics
    accu, prec, rec, f1 = [], [], [], []
    
    # models
    mdls = []
    
    # predictions
    predictions = []
    truths = []
    
    # iterate over the masks
    for mTrn, mVal, mTst in zip(train_masks, validate_masks, test_masks):
        dtrain = xgb.DMatrix(feats.loc[mTrn], label=labels.loc[mTrn])
        dval = xgb.DMatrix(feats.loc[mVal], label=labels.loc[mVal])
        if test:
            dtest = xgb.DMatrix(feats.loc[mTst], label=labels.loc[mTst])
        
        eval_list = [(dtrain, 'train')]
        bst = xgb.train(params, dtrain, 80, eval_list, verbose_eval=False)
        
        if test:
            y_pred_ = bst.predict(dtest)
            y_pred =  y_pred_ > 0.5
            y_true = labels.loc[mTst]
        else:
            y_pred_ = bst.predict(dval)
            y_pred = y_pred_ > 0.5
            y_true = labels.loc[mVal]
        
        accu.append(accuracy_score(y_true, y_pred, normalize=True))
        prec.append(precision_score(y_true, y_pred))
        rec.append(recall_score(y_true, y_pred))
        f1.append(f1_score(y_true, y_pred))
        
        mdls.append(bst)
        predictions.append(y_pred_)
        truths.append(y_true)
        
    print(f'Average Accuracy: {np.mean(accu):.2f} ({np.std(accu):.2f})')
    print(f'Average Precision: {np.mean(prec):.2f} ({np.std(prec):.2f})')
    print(f'Average Recall: {np.mean(rec):.2f} ({np.std(rec):.2f})')
    print(f'Average F1 Score: {np.mean(f1):.2f} ({np.std(f1):.2f})')
        
    return mdls, feats.columns, predictions, truths, accu, prec, rec, f1

In [4]:
data = pd.read_hdf('../feature_exploration/features.h5', key='no_preprocessing')

In [5]:
# test with stairs as positive class
mask = (data.Activity == 'stairs-ascending') | (data.Activity == 'stairs-descending')
data.loc[mask, 'Label'] = 1

In [6]:
# get the subjects for which LOSO actually makes sense: those with multiple activities (ie more than just walking)
gbc = data.groupby(['Subject', 'Activity'], as_index=False).count()
loso_subjects = [i for i in gbc.Subject.unique() if gbc.loc[gbc.Subject == i].shape[0] > 3]

random.seed(5)  # fix the generation so that its the same every time
random.shuffle(loso_subjects)

training_masks = []
validation_masks = []
testing_masks = []

for i in range(0, len(loso_subjects), 4):
    tr_m = np.ones(data.shape[0], dtype='bool')
    v_m = np.zeros(data.shape[0], dtype='bool')
    te_m = np.zeros(data.shape[0], dtype='bool')
    
    for j in range(4):
        tr_m &= (data.Subject != loso_subjects[i+j]).values
    for j in range(2):
        v_m |= (data.Subject == loso_subjects[i+j]).values
    for j in range(2):
        te_m |= (data.Subject == loso_subjects[i+j+2]).values
    
    training_masks.append(tr_m)
    validation_masks.append(v_m)
    testing_masks.append(te_m)

masks = (training_masks, validation_masks, testing_masks)

In [7]:
# drop the least important features (based on PPScore)
# drop all values with predictive score < 0.3, and "duplicate" features
data_subset = data.drop([
    '_rangecountpercentage',
    '_skewness',
    '_linearslope',
    '_kurtosis',
    '_complexityinvariantdistance',
    '_spectralflatness',
    '_autocorrelation',
    '_powerspectralsum',
    '_ratiobeyondrsigma',
    '_signalentropy',
    '_dominantfrequencyvalue',
    '_permutationentropy',
    '_meancrossrate',
    '_spectralentropy',
    '_detailpower',  # less than 0.3, duplicated by DetailPowerRatio
    '_jerkmetric',  # duplicated by dimensionlessJerk
    '_stddev'  # duplicated by RMS
], axis=1)

In [16]:
cvr = pd.read_csv('xgbrf_cv_results_incl_stairs.csv', index_col=0)
params = ast.literal_eval(
    cvr.sort_values('rank_test_F1').params.values[0]
)

In [21]:
mdls, feats, preds, truths, *metrics = xgb_LOSO_runner(
    data,
    params,
    *(training_masks, validation_masks, testing_masks),
    test=False
)

[10:53:57] WARNING: /workspace/src/learner.cc:480: 
Parameters: { importance_type, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:53:58] WARNING: /workspace/src/learner.cc:480: 
Parameters: { importance_type, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:53:59] WARNING: /workspace/src/learner.cc:480: 
Parameters: { importance_type, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip throug

In [22]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve, average_precision_score

In [23]:
f, (ax, ax1) = plt.subplots(ncols=2, figsize=(10, 6), sharey=True)

opt_trsh = []

i = 1
for p_, t_ in zip(preds, truths):
    fpr, tpr, thrsh = roc_curve(t_, p_)
    
    # compute optimal threshold based on Youdin's index
    dist = tpr - fpr
    
    opt_trsh.append(thrsh[np.argmax(dist)])
    
    ax.plot(fpr, tpr, label=f'Fold{i:3d}: {auc(fpr, tpr):.2f}')
    ax1.plot(thrsh, dist, label=f'Fold{i:3d}: {opt_trsh[-1]:.2f}')
    i += 1

ax.legend(loc='best')
ax1.legend(loc=3)

ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax1.axvline(0.5, color='navy', lw=2, linestyle='--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax1.set_xlim([0.0, 1.0])

ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

f.tight_layout()

print(np.mean(opt_trsh), np.median(opt_trsh))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.51660734 0.5132371


In [25]:
f, ax = plt.subplots(figsize=(10, 6), sharey=True)

i = 1
for p_, t_ in zip(preds, truths):
    precision, recall, thrsh = precision_recall_curve(t_, p_)
    ap = average_precision_score(t_, p_)
    
    ax.plot(recall, precision, label=f'Fold{i:3d}: {ap:.2f}')
    i += 1
    
f_scores = np.linspace(0.1, 0.9, num=5)
lines = []
labels = []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    l, = ax.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
    ax.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

ax.legend(loc='best')

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])

ax.set_xlabel('Recall')
ax.set_ylabel('Precision')

f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
fp_acts = []
fp_fold = []
fp_certainty = []

i = 1
for mask, pred, truth in zip(validation_masks, preds, truths):
    false_pos = mask.copy()
    false_pos[false_pos] &= (pred > 0.6) & (truth == 0)
    
    fp_acts.extend(data.Activity.loc[false_pos].tolist())
    fp_fold.extend([i] * false_pos.sum())
    fp_certainty.extend(pred[false_pos[mask]])
    
    i += 1

fp = pd.DataFrame(data={
    'Activity': fp_acts,
    'Fold': fp_fold, 
    'Certainty': fp_certainty
})

In [62]:
f, ax = plt.subplots(figsize=(9, 5))

sns.boxplot(x='Activity', y='Certainty', hue='Fold', data=fp, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

ax.set_title('False Positive Certainty Distribution')

f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
fp.groupby('Activity').count().sort_values('Fold')

,Fold,Certainty
Activity,,
washing-dishes,2,2
elevator-descending,4,4
standing-assisted,4,4
vacuuming,5,5
running-treadmill,8,8
standing,12,12
running,40,40
cycling-100W,161,161
sweeping,163,163


In [65]:
fn_acts = []
fn_fold = []
fn_certainty = []

i = 1
for mask, pred, truth in zip(validation_masks, preds, truths):
    false_neg = mask.copy()
    false_neg[false_neg] &= (pred < 0.6) & truth
    
    fn_acts.extend(data.Activity.loc[false_neg].tolist())
    fn_fold.extend([i] * false_neg.sum())
    fn_certainty.extend(pred[false_neg[mask]])
    
    i += 1

fn = pd.DataFrame(data={
    'Activity': fn_acts,
    'Fold': fn_fold, 
    'Certainty': fn_certainty
})

In [71]:
f, ax = plt.subplots(figsize=(9, 5))

sns.boxplot(x='Activity', y='Certainty', hue='Fold', data=fn, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

ax.set_title('False Negative Certainty Distribution')

f.tight_layout()

<ipython-input-71-a02e4b0dbf28>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, ax = plt.subplots(figsize=(9, 5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
fn.groupby('Activity').count()

,Fold,Certainty
Activity,,
stairs-ascending,3157,3157
stairs-descending,2952,2952
walking,3263,3263
walking-impaired,2087,2087
walking-left,1062,1062
walking-right,1180,1180
